In [1]:
import pickle
import cobra
import lftc
import igraph
import pandas as pd
import re

In [2]:
results = pickle.load(open('test_data/results_e_coli.p', 'rb'))
results.sort(key=lambda r: r[1])
print(results[0][1])
results = results[0][0]
len(results)

1.19855984


141

In [3]:
with open('test_data/REACTIONSwt5h.txt') as f:
    allLines = f.readlines()
reactionLines = list(filter(lambda l: re.match('^\w', l), allLines))
coreReactionNames = set([re.sub('\s.*$', '', l) for l in reactionLines])
coreReactionNames = set([re.sub('[\(,\)]', '_', l) for l in coreReactionNames])

print(str(len(coreReactionNames)) + ' core reactions in transitions file')

127 core reactions in transitions file


In [4]:
model = cobra.io.read_sbml_model('test_data/EciJR904TKs_sbml3.xml')
feed='EX_glc_e_'

In [5]:
len(results)/len(model.reactions)

0.1309192200557103

In [26]:
model_new = cobra.io.read_sbml_model('test_data/iJR904.xml')

In [33]:
# show all of the newly added reactions
newReactions = set(results).difference(coreReactionNames)
for reaction in newReactions:
    rxn = model.reactions.get_by_id(reaction)
    print(rxn)
    metabolites = {m.id for m in rxn.products}
    metabolites.update({m.id for m in rxn.reactants})
    for m in metabolites:
        if m in [m.id for m in model_new.metabolites]:
            print(m, model_new.metabolites.get_by_id(m).formula)

ANS: chor_c + gln_L_c --> anth_c + glu_L_c + h_c + pyr_c
anth_c C7H6NO2
h_c H
chor_c C10H8O6
pyr_c C3H3O3
IMPD: h2o_c + imp_c + nad_c --> h_c + nadh_c + xmp_c
nadh_c C21H27N7O14P2
h2o_c H2O
h_c H
xmp_c C10H11N4O9P
nad_c C21H26N7O14P2
imp_c C10H11N4O8P
THDPS: h2o_c + succoa_c + thdp_c --> coa_c + sl2a6o_c
thdp_c C7H7NO4
succoa_c C25H35N7O19P3S
sl2a6o_c C11H12NO8
coa_c C21H32N7O16P3S
h2o_c H2O
GARFT: 10fthf_c + gar_c <=> fgam_c + h_c + thf_c
gar_c C7H14N2O8P
thf_c C19H21N7O6
fgam_c C8H13N2O9P
10fthf_c C20H21N7O7
h_c H
PFL: coa_c + pyr_c --> accoa_c + for_c
pyr_c C3H3O3
coa_c C21H32N7O16P3S
accoa_c C23H34N7O17P3S
for_c CH1O2
ADCS: chor_c + gln_L_c --> 4adcho_c + glu_L_c
4adcho_c C10H10NO5
chor_c C10H8O6
GTPCI: gtp_c + h2o_c --> ahdt_c + for_c
ahdt_c C9H12N5O13P3
gtp_c C10H12N5O14P3
h2o_c H2O
for_c CH1O2
GF6PTA: f6p_c + gln_L_c --> gam6p_c + glu_L_c
gam6p_c C6H13NO8P
f6p_c C6H11O9P
DHDPRy: 23dhdp_c + h_c + nadph_c --> nadp_c + thdp_c
h_c H
nadp_c C21H25N7O17P3
23dhdp_c C7H5NO4
nadph_c C21H

In [7]:
sortedMetabolites = pd.DataFrame([(m.id, len(m.reactions)) for m in model.metabolites]).sort_values(1,ascending=False)
set(sortedMetabolites[0:0][0]).difference(lftc.currencyMetabolites)

set()

In [8]:
len(model.reactions)

1077

In [9]:
modelGraph = igraph.Graph(directed=False)
currencyMetabolites=lftc.currencyMetabolites
modelGraph.add_vertex(name=feed)
newCoreReactionNames = set()
unexploredReactionNames = {feed} # this starts with just the feed name
while len(unexploredReactionNames) > 0:
    reactionName = unexploredReactionNames.pop()
    newCoreReactionNames.update([reactionName])
    # modelGraph.add_vertex(name=reactionName)
    reaction = model.reactions.get_by_id(reactionName)
    connectedMetabolites = {m.id for m in reaction.reactants}
    connectedMetabolites.update({m.id for m in reaction.products})
    connectedMetabolites = connectedMetabolites.difference(currencyMetabolites)
    sourceNode = modelGraph.vs["name"].index(reactionName)
    
    for metaboliteName in connectedMetabolites:
        metabolite = model.metabolites.get_by_id(metaboliteName)
        connectedReactionNames = {r.id for r in metabolite.reactions}
        # connectedReactionNames = connectedReactionNames.intersection(coreReactionNames)
        newReactionsThisLoop = connectedReactionNames.difference(newCoreReactionNames)
        unexploredReactionNames.update(newReactionsThisLoop)
        newCoreReactionNames.update(newReactionsThisLoop)
        for newReactionName in connectedReactionNames:
            # print("connected to", reactionName, len(newReactionName))
            if not newReactionName in modelGraph.vs["name"]:
                modelGraph.add_vertex(name=newReactionName)
            
            targetNode = modelGraph.vs["name"].index(newReactionName)
            if not modelGraph.are_connected(sourceNode, targetNode):
                modelGraph.add_edge(source=sourceNode, target=targetNode)

In [10]:
print(modelGraph)

IGRAPH UN-- 1028 8212 --
+ attr: name (v)
+ edges (vertex names):
     EX_glc_e_ -- EX_glc_e_, EX_glc_e_, GLCt2, TREHe, GLCpts, GLCDe
         GLCt2 -- EX_glc_e_, GLCt2, GLCt2, TREHe, GLCpts, GLCDe, AMALT2, G1PP,
GALS3, LACZ, MLTG2, MLTG4, MLTG1, AMALT4, TREH, HEX1, MLTG3, AMALT3, XYLI2i,
TRE6PH, AMALT1, MLTG5
         TREHe -- EX_glc_e_, GLCt2, TREHe, TREHe, GLCpts, GLCDe, TREpts,
TMAOR2e, DMSOR2e, DMSOR1e, TMAOR1e, NADDPe, EX_h2o_e_, H2Ot, EX_tre_e_
        GLCpts -- EX_glc_e_, GLCt2, TREHe, GLCpts, GLCpts, GLCDe, HEX1,
TRE6PH, PGMT, TRE6PS, PGI, FFSD, G6PDH2r, G6Pt6_2, AB6PGH, ENO, SBTpts, PPCK,
ACMANApts, TREpts, PPC, MANpts, DHAPT, PYK, MNLpts, MALTpts, DDPA, KDOPS,
GAMpts, SUCpts, PSCVT, PPS, UAGCVT, ACGApts, FRUpts2, FRUpts, GALTpts, CHRPL,
ME2, ACLS, LDH_D2, SERD_L, DHDPS, ALATA_L2, VPAMT, LDH_D, SHCHCS2, ACNML,
PYRt2r, PFL, DAAD, CYSTL, PDH, DDPGALA, EDA, ADCL, ME1, DXPS, TRPAS2, HOPNTAL,
POX, ALATA_L, L_LACD3, L_LACD2, ANS, GLCRAL, ICHORT, SERD_D, ACHBS, ALATA_D2,
MCITL2, CYS

In [11]:
modelGraph.is_connected(mode=igraph.WEAK)

True

In [12]:
for reactionName in results:
    modelGraph.vs[modelGraph.vs["name"].index(reactionName)]['class'] = 'newcore'

for reactionName in coreReactionNames:
    modelGraph.vs[modelGraph.vs["name"].index(reactionName)]['class'] = 'oldcore'
    


In [13]:
modelGraph.save('test_data/graph.graphml', 'graphml')